Loasd JSON file into a dataframe

In [2]:
import pandas as pd
import json
 
data = pd.read_json('categorized-comments.jsonl', lines=True)
df = pd.DataFrame(data)
df.head(5)

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


Convert all text to lower case letters

In [2]:
df_cat = df['cat']
df_cat = pd.DataFrame(df_cat)
df= df['txt'].str.lower()
df = pd.DataFrame(df)
df.head(10)


,txt
0,barely better than gabbert? he was significant...
1,fuck the ducks and the angels! but welcome to ...
2,should have drafted more wrs.\n\n- matt millen...
3,[done](https://i.imgur.com/2yz90pm.jpg)
4,no!! noo!!!!!
5,ding dong the kaepers gone!!!!!! yes!!!! frida...
6,yup\n\nthat would be best case scenario. still...
7,i think larry kruger made a good point on knbr...
8,this is great to have two well-regarded rb coa...
9,7-9 next season confirmed.


Remove all punctuation from text

In [3]:
df["txt"] = df['txt'].str.replace('[^\w\s]','')
df.head(5)

<ipython-input-3-07822b49f9ce>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["txt"] = df['txt'].str.replace('[^\w\s]','')


,txt
0,barely better than gabbert he was significantl...
1,fuck the ducks and the angels but welcome to a...
2,should have drafted more wrs\n\n matt millen p...
3,donehttpsiimgurcom2yz90pmjpg
4,no noo


Remove stop words from text

In [5]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['txt'] = df["txt"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df

,txt
0,barely better gabbert significantly better yea...
1,fuck ducks angels welcome new niners fans
2,drafted wrs matt millen probably
3,donehttpsiimgurcom2yz90pmjpg
4,noo
...,...
606469,youre willing splash get 35 hard drive adapter...
606470,gtany chance install entire contents disc driv...
606471,probably happened
606472,think disappointment came delay ps version rea...


 Apply NLTK’s PorterStemmer.

In [6]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['txt'] = df['txt'].apply(stem_sentences)
df

,txt
0,bare better gabbert significantli better year ...
1,fuck duck angel welcom new niner fan
2,draft wr matt millen probabl
3,donehttpsiimgurcom2yz90pmjpg
4,noo
...,...
606469,your will splash get 35 hard drive adapt 6tb h...
606470,gtani chanc instal entir content disc drive al...
606471,probabl happen
606472,think disappoint came delay ps version realli ...


In [7]:
# Change diagnosis name to target the better represent the outcome of the model data
df_cat = df_cat.rename(columns={'cat': 'target'})
#Check to see what the balance is between Benign and Malignant tumor diagnosis.
df_cat['target'].value_counts(dropna=False)

video_games               435540
sports                    145823
science_and_technology     25111
Name: target, dtype: int64

In [43]:
df_cat = df_cat.replace({'target': {'video_games': 0, 'sports': 1,'science_and_technology':2}})
df_target = df_cat['target']
df_target

0         1
1         1
2         1
3         1
4         1
         ..
606469    0
606470    0
606471    0
606472    0
606473    0
Name: target, Length: 606474, dtype: int64

In [44]:
df_new = pd.concat([df_target,df],axis=1)
df_new

,target,txt
0,1,bare better gabbert significantli better year ...
1,1,fuck duck angel welcom new niner fan
2,1,draft wr matt millen probabl
3,1,donehttpsiimgurcom2yz90pmjpg
4,1,noo
...,...,...
606469,0,your will splash get 35 hard drive adapt 6tb h...
606470,0,gtani chanc instal entir content disc drive al...
606471,0,probabl happen
606472,0,think disappoint came delay ps version realli ...


In [180]:
from sklearn.model_selection import train_test_split

target_column = ['target'] 
predictors = list(set(list(df_new.columns))-set(target_column))

X = df_new[predictors]
y = df_new[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

(424531, 1)
(181943, 1)


,txt
275858,play game onlin someth specif requir internet ...
232205,httpswwwfutbincom17squad1736993
601114,liter right
455302,good call thank remind
414248,im buy switch addit joycon pro control zelda c...
...,...
29618,go blue
340146,pleas tri trade thread check muthead forum
427175,saw one edit boss come back php
437231,target best bet take instor preorder far im aw...


In [182]:
def Ngram_Vectorizer(reviews_train, reviews_test):
    tfidf = TfidfVectorizer(max_features=500,lowercase=False)
    tfidf.fit(reviews_train)
    feature_names = tfidf.get_feature_names()
    reviews_train = tfidf.transform(reviews_train).toarray()
    reviews_test = tfidf.transform(reviews_test).toarray()
    return reviews_train, reviews_test

In [183]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create object
tfidf = TfidfVectorizer(lowercase=False)
  
# get tf-df values
X_train_tfidf,X_test_tfidf = Ngram_Vectorizer(X_train['txt'], X_test['txt'])

In [156]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
# Use MLP neural networkk classifier with 500,150 layers and 20 max iterations
mlp = MLPClassifier(hidden_layer_sizes=([500,150]),verbose=True,max_iter=20)
mlp.fit(X_train_tfidf,y_train.ravel())

#predict_train = mlp.predict(X_train_tfidf[:,1])
#predict_test = mlp.predict(X_test_tfidf[:,1])
predict_train = mlp.predict(X_train_tfidf)
predict_test = mlp.predict(X_test_tfidf)

Iteration 1, loss = 0.57301902
Iteration 2, loss = 0.54388044
Iteration 3, loss = 0.52078308
Iteration 4, loss = 0.49397843
Iteration 5, loss = 0.46594771
Iteration 6, loss = 0.43901589
Iteration 7, loss = 0.41443792
Iteration 8, loss = 0.39281050
Iteration 9, loss = 0.37552116
Iteration 10, loss = 0.36033432
Iteration 11, loss = 0.34839983
Iteration 12, loss = 0.33812854
Iteration 13, loss = 0.33058943
Iteration 14, loss = 0.32104865
Iteration 15, loss = 0.31457994
Iteration 16, loss = 0.30820751
Iteration 17, loss = 0.30267449
Iteration 18, loss = 0.29930372
Iteration 19, loss = 0.29394953
Iteration 20, loss = 0.28945521


C:\Users\dan35\anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [157]:
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

[[298416   6037    424]
 [ 36610  65360    106]
 [  5005    269  12304]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.93    304877
           1       0.91      0.64      0.75    102076
           2       0.96      0.70      0.81     17578

    accuracy                           0.89    424531
   macro avg       0.92      0.77      0.83    424531
weighted avg       0.89      0.89      0.88    424531



In [193]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

[[113985  14483   2195]
 [ 27679  15499    569]
 [  4493    704   2336]]
              precision    recall  f1-score   support

           0       0.78      0.87      0.82    130663
           1       0.51      0.35      0.42     43747
           2       0.46      0.31      0.37      7533

    accuracy                           0.72    181943
   macro avg       0.58      0.51      0.54    181943
weighted avg       0.70      0.72      0.71    181943



# Using Keras to do a Neural Network

In [1]:
# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

In [185]:
# one hot encode outputs
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test)

count_classes = y_test1.shape[1]
print(count_classes)



3


In [164]:
model = Sequential()
model.add(Dense(500, activation='relu', input_shape=(500,)))
model.add(Dense(150, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [165]:
print(X_train_tfidf.shape)
print(y_train1.shape)

(424531, 500)
(424531, 3)


In [166]:
# build the model
model.fit(X_train_tfidf, y_train1, epochs=50,batch_size=120)

Epoch 1/50
3538/3538 [==============================] - 11s 3ms/step - loss: 0.5734 - accuracy: 0.7551
Epoch 2/50
3538/3538 [==============================] - 8s 2ms/step - loss: 0.5459 - accuracy: 0.7665
Epoch 3/50
3538/3538 [==============================] - 9s 2ms/step - loss: 0.5235 - accuracy: 0.7766
Epoch 4/50
3538/3538 [==============================] - 7s 2ms/step - loss: 0.4967 - accuracy: 0.7878
Epoch 5/50
3538/3538 [==============================] - 9s 2ms/step - loss: 0.4673 - accuracy: 0.8002
Epoch 6/50
3538/3538 [==============================] - 8s 2ms/step - loss: 0.4379 - accuracy: 0.8115
Epoch 7/50
3538/3538 [==============================] - 11s 3ms/step - loss: 0.4113 - accuracy: 0.8220
Epoch 8/50
3538/3538 [==============================] - 8s 2ms/step - loss: 0.3890 - accuracy: 0.8304
Epoch 9/50
3538/3538 [==============================] - 8s 2ms/step - loss: 0.3703 - accuracy: 0.8378
Epoch 10/50
3538/3538 [==============================] - 8s 2ms/step - loss: 0.3

In [177]:
score = model.evaluate(X_train_tfidf, y_train1, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.2188085913658142
Test accuracy: 0.9053167104721069


In [186]:
score1 = model.evaluate(X_test_tfidf, y_test1, verbose = 0) 

print('Test loss:', score1[0]) 
print('Test accuracy:', score1[1])

Test loss: 2.6663546562194824
Test accuracy: 0.7230781316757202


# Make a Neural Network Image Classifier

In [73]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [74]:
# Set that the color channel will be first
K.set_image_data_format("channels_last")

# Set seed
np.random.seed(0)

# Set image information
channels = 1
height = 28
width = 28

#Load data and target from MNIST data
(data_train, target_train),(data_test, target_test) = mnist.load_data()


In [75]:
# Reshape training image data into features
data_train = data_train.reshape(data_train.shape[0],height, width,channels)

data_test = data_test.reshape(data_test.shape[0],height, width,channels)

In [76]:
# Rescale pixel intensity to be between 0 and 1
features_train = data_train/255
features_test = data_test/255

In [77]:
# One hot encode target
target_train = np_utils.to_categorical(target_train)
target_test = np_utils.to_categorical(target_test)
number_of_classes = target_test.shape[1]

In [78]:
# Start Neural Network
network = Sequential()

In [79]:
# Add convolutional layer with 64 filters, a 5x5 window, and ReLU activation function
network.add(Conv2D(filters=64,
                  kernel_size=(5,5),
                  input_shape=(height,width,channels),
                  activation='relu'))

In [80]:
# Add max pooling layer with a 2x2 window
network.add(MaxPooling2D(pool_size=(2,2)))

In [81]:
# Add layer to flatten input
network.add(Flatten())

In [82]:
# Add fully connected layer with a softmax activation function
network.add(Dense(number_of_classes,activation='softmax'))

In [83]:
# Compile network
network.compile(loss='categorical_crossentropy',
               optimizer='rmsprop',
               metrics=['accuracy'])

In [84]:
# Train Neural Network
network.fit(features_train,
           target_train,
           epochs=2,
           verbose=1,
           batch_size=1000,
           validation_data=(features_test,target_test))



Epoch 1/2
60/60 [==============================] - 16s 263ms/step - loss: 0.5935 - accuracy: 0.8374 - val_loss: 0.2787 - val_accuracy: 0.9212
Epoch 2/2
60/60 [==============================] - 15s 246ms/step - loss: 0.2317 - accuracy: 0.9321 - val_loss: 0.1774 - val_accuracy: 0.9496


In [86]:
score = network.evaluate(features_test, target_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.1773950308561325
Test accuracy: 0.9495999813079834
